In [17]:
# !pip install -q unstructured

In [18]:
# !pip install -q pdfminer

In [19]:
# !pip install --upgrade -q unstructured

In [20]:
import os
import uuid
import json
import numpy as np
from PIL import Image
from pydantic import BaseModel
from sqlalchemy import create_engine, MetaData

from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from pprint import pprint

from sentence_transformers import SentenceTransformer

from langchain.utilities import SQLDatabase
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain_ollama import OllamaEmbeddings
from langchain.schema import Document
from langchain.vectorstores import Qdrant

from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, VectorParams, Distance

from dotenv import load_dotenv
load_dotenv(override=True)

import warnings
warnings.filterwarnings("ignore")

# Process User Manual PDF and Save as JSON

In [21]:
# file_path = "../docs/Sitearound User Manual - 2025.pdf"

# loader = PyPDFLoader(file_path,
#                      mode="page")

In [22]:
# docs = []
# for doc in loader.lazy_load():
#     docs.append(doc)

In [23]:
# user_manual_dict = {
#     'title': [],
#     'content': [],
# }

# signle_page_content = ""
# titles = []
# n = 1
# for i, page in enumerate(docs):
#     if i < 5:  # Skip first 5 pages
#         continue

#     content = page.page_content.split("\n")

#     # Remove the first 4 lines (header)
#     content = content[4::]

#     # Strip leading/trailing whitespace from each line
#     content = [line.strip() for line in content]
    
#     for c in content:
#         if f"Chapter {n}" in c:
#             titles.append(c)
#             signle_page_content += f"\nPage: "
#             n += 1

#     signle_page_content += " ".join(content)


In [24]:
# from typing import List


# data: List[dict] = []
# contents = signle_page_content.split("Page: ")[1:]

# for title, content in zip(titles, contents):
#     data.append({
#         "title": title,
#         "content": content
#     })


In [25]:
# import json

# json_object = json.dumps(data, indent=4)
# with open("../docs/user_manual.json", "w") as outfile:
#     outfile.write(json_object)

# Prepare data for vector store

In [26]:
embeddings = OllamaEmbeddings(model="bge-m3:latest")

In [27]:
data = json.load(open("../docs/user_manual.json"))
data

[{'title': 'Chapter 1 Corporate',
  'content': 'Chapter 1 Corporate Sitearound has two levels: Corporate and Project. The Project level grants users access to individual projects, while the Corporate level enables leaders to access multiple projects and create new ones.  1.1 Main Menu At the Corporate level, you can do everything available at the Project level, plus access all your projects within the account and manage users. This includes setting up permission templates to control user access to different tools in Sitearound.1.2 Directory Overview 1. To view and manage users and companies in Sitearound account, click a “Directory” option on the side menu.   2. Users can be viewed by selecting either under the “Users” or “Company” tab at the top of the page.1.3 Create New User 1. To add a new user, click a “Directory” option on the side menu.   2. In a “Directory” page, click “Create User” button on the top right of the page.3. Enter user information, then click a “Create” button on t

In [28]:
points = []
for i, d in enumerate(data):
    title = d['title']
    content = d['content']
    point = PointStruct(
        id=i,
        vector=embeddings.embed_query(content),
        payload={"title": " ".join(title.split(" ")[2::]), "content": content}
    )
    points.append(point)

# Embeded

In [29]:
qdrant_client  = QdrantClient(url=os.getenv("QDRANT_URL"))

In [30]:
COL_NAME = "CM-User-Manual"
if not qdrant_client.collection_exists(collection_name=COL_NAME): 
    qdrant_client.create_collection(
        collection_name=COL_NAME,
        vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
        # optional: tune shards, replication_factor, etc.
    )
else:
    print("Collection already created")

In [31]:
qdrant_client.upsert(collection_name=COL_NAME, points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)